In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

In [5]:
numerical_features = [col for col in X.columns if df[col].dtype!= 'O' ]
categorical_features = [col for col in X.columns if df[col].dtype== 'O' ]
numerical_features, categorical_features

(['CreditScore',
  'Age',
  'Tenure',
  'Balance',
  'NumOfProducts',
  'HasCrCard',
  'IsActiveMember',
  'Exited'],
 ['Geography', 'Gender'])

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[  
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [9]:
import pickle

with open('salarypreprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [26]:
model = Sequential( [
    Dense(64, activation ='relu', input_shape=(X_train_processed.shape[1],)),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'linear')
])

In [27]:
model.compile(optimizer='adam',loss='mean_absolute_error', metrics=['mae'])

In [29]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

logs_dir = "logsSalary/fit/"+ datetime.datetime.now().strftime("Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss',patience=30, restore_best_weights=True)

In [30]:
history = model.fit(X_train_processed, y_train,
                    validation_data=[X_test_processed, y_test],
                    epochs=100,
                    callbacks=[early_stopping, tensorboard_callback]
                   )

Epoch 1/100
250/250 [==============================] - 5s 10ms/step - loss: 100358.2891 - mae: 100358.2891 - val_loss: 98432.3672 - val_mae: 98432.3672
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 99287.7266 - mae: 99287.7266 - val_loss: 96271.6719 - val_mae: 96271.6719
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 95579.0859 - mae: 95579.0859 - val_loss: 90894.5156 - val_mae: 90894.5156
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 88568.1328 - mae: 88568.1328 - val_loss: 82461.5938 - val_mae: 82461.5938
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 79143.4922 - mae: 79143.4922 - val_loss: 72512.9141 - val_mae: 72512.9141
Epoch 6/100
250/250 [==============================] - 3s 11ms/step - loss: 69300.8984 - mae: 69300.8984 - val_loss: 63415.5508 - val_mae: 63415.5508
Epoch 7/100
250/250 [==============================] - 2s 8ms/step - loss: 61473.8477 - mae: 61473.847

In [32]:
%load_ext tensorboard
%tensorboard --logdir logsSalary/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6012 (pid 13712), started 0:00:58 ago. (Use '!kill 13712' to kill it.)

In [33]:
test_loss, test_mae = model.evaluate(X_test_processed, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 4ms/step - loss: 50046.2109 - mae: 50046.2109
Test MAE: 50046.2109375


In [34]:
model.save('salary_model.h5')

e:\Data Science\End to End Deep Learning Project using ANN\annclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
